In [12]:
import os
import fnmatch

In [13]:
def get_ignore_patterns(start_path):
    patterns = []
    
    # Read all .gitignore files
    for root, dirs, files in os.walk(start_path):
        if '.gitignore' in files:
            with open(os.path.join(root, '.gitignore'), 'r') as f:
                for line in f.readlines():
                    line = line.strip()  # remove whitespace
                    if line and not line.startswith('#'):  # ignore empty lines and comments
                        # Add the root to the pattern to correctly match subdirectories
                        pattern = os.path.join(root, line)
                        patterns.append(pattern)    
    patterns.append('./*.ipynb')
    return patterns

def get_file_structure(start_path, ignore_paths):
    file_structure = {}
    ignore_patterns = get_ignore_patterns(start_path)
    for root, dirs, files in os.walk(start_path, topdown=True):
        dirs[:] = [d for d in dirs if d not in ignore_paths]  # remove the ignored directories from dirs
        files[:] = [f for f in files if f not in ignore_paths]  # remove the ignored directories from dirs        
        for file in files:
            path = os.path.join(root, file)
            if not any(fnmatch.fnmatch(path, pattern) for pattern in ignore_patterns):  # only add if not in ignore list
                current_level = file_structure
                path_parts = path.split(os.sep)[1:]  # split the path into parts and ignore the first part (the start_path)
                for part in path_parts[:-1]:  # for all parts except the last one (the file itself)
                    current_level = current_level.setdefault(part, {})
                current_level[path_parts[-1]] = None  # set the file as a key with value None
    return file_structure

def print_file_structure(structure, descriptions, indent_level=0):
    indent = '  ' * indent_level  # two spaces per indent level
    result = ""
    for name, substructure in structure.items():
        if substructure is None:  # it's a file
            description = descriptions.get(name, '')
            if description == '':
                result += "{}- {}\n".format(indent, name)
#                 print("{}- {}".format(indent, name))
            else:
                result += "{}- {}: {}\n".format(indent, name, description)
#                 print("{}- {}: {}".format(indent, name, description))
        else:  # it's a directory
            result += f'{indent}- {name}/\n'
#             print(f'{indent}- {name}/')
            result += print_file_structure(substructure, descriptions, indent_level+2)
    return result


In [26]:
description = {
    "main.py" : "a Flask server Script that handles user registration, login, and interactive chat sessions using SocketIO, with a chatbot utilizing GPT for generating responses.",
    "simple_gpt_chatbot.py" : "Interfaces with OpenAI's GPT-3.5 model to generate responses for a chatbot, using the historical data of a chat session retrieved from a database.",
    "model.py" : "Defines the structure of the database tables for the chat application, including Users, ChatSessions, ChatMessages, Chatbots, and the relationships between them.",
    "database.py" : "Initializes an SQLAlchemy database instance for the application to interact with the database.",
    "App.js" : "A React-based client-side application script managing user authentication, registration, chat sessions using Socket.IO, and component rendering according to user authentication status.",
    "ChatPanel.js" : "A React component managing the real-time chat interface, handling message sending/receiving via Socket.IO, maintaining chat session state, and rendering the chat messages and input field." ,
    "Input.js" : "A React component that renders a message input field and send button, handling message submission and maintaining the state of the typed message.",
    "Login.js" : " A React component that handles user login functionality. It includes form inputs for email and password, validation logic, error handling, language selection, and navigation",
    "Message.js" : "A React component that renders a chat message, including the profile photo, username (for remote messages), and the content of the message.",
    "Registration.js" : "A React component that renders a registration form with input fields for ID and passcode, handles form submission, performs registration request to the server using axios, and subsequently performs login.",
    "SocketContext.js" : "A React context that provides a Socket.IO socket instance to its child components.",
    "index.js" : "The entry point of the React application, rendering the root component (App) into the DOM, enabling React Strict Mode, and optionally measuring performance using the reportWebVitals function.",    
    "ChatSessionList.js": "A React component, ChatSessionList, that fetches and displays a list of chat sessions from a server, allowing a user to select a specific chat session",
    "ChatPage.js": "A React component that is parent of ChatPanel and SidePanel. It represents the main page of a chat application. It includes a chat panel and a side panel for managing chat sessions. The component establishes a socket connection, handles user authentication, and handles UI interactions such as resizing and toggling the side panel. The component also provides the socket context to its child components.",
    "Signup.js": "A React component that handles user signup functionality. It includes form inputs for email and password, validation logic, error handling, language selection, and navigation.",
    "SidePanel.js": "A React component that represents the side panel of a chat application. It includes features such as displaying chat sessions, fetching and displaying the user's username, starting a new chat, and logging out."
    
    
    
    
}

In [27]:
start_path = "../"
ignore_files = ["copilot", "venv", "node_modules", "build", ".git", "logo.svg", "public", "Procfile", ".ipynb_checkpoints", "LICENSE", "README.md", "copilot.ipynb", ".gitignore", ".env", ".DS_Store"]
# "package-lock.json", "package.json"
file_structure = get_file_structure(start_path, ignore_files)
file_descriptions = print_file_structure(file_structure, description, 0)

In [24]:
project_name = "nplace_webchat"


role = """
Your role is developer and product designer copilot. We collaborate on project: {}.
""".format(project_name)


project_overview = """
Develop a website enabling user registration/login and chat with AI-powered chatbots.
Users can create, rename, and delete chat sessions.
When creating a chat session, a user can choose from a list of predefined themes, such as dragon and dungeon, mafia, etc.
Depending on the theme, session may feature one or multiple chatbots, but restricted to a single user.
Future work will explore instant messaging between multiple users.

"""


steps_taken = """
- Simple frontend using React with Messenger-like UI, backend using Flask
- Code on Github, hosted on Railway, no CICD 
- Simple user authentication using JWT
- PostgreSQL DB to store user credentials and chat data
- Simple chatbot using OpenAI GPT API
- Basic chat session management that allow creating new or selecting previous chat seesions
"""


prompt_1 = f"""{role}
Project overview {project_overview}
Steps taken {steps_taken}
Show that you understand the overview and current status. Next prompt, I will brief you details about current task."""


print(prompt_1)


Your role is developer and product designer copilot. We collaborate on project: nplace_webchat.

Project overview 
Develop a website enabling user registration/login and chat with AI-powered chatbots.
Users can create, rename, and delete chat sessions.
When creating a chat session, a user can choose from a list of predefined themes, such as dragon and dungeon, mafia, etc.
Depending on the theme, session may feature one or multiple chatbots, but restricted to a single user.
Future work will explore instant messaging between multiple users.


Steps taken 
- Simple frontend using React with Messenger-like UI, backend using Flask
- Code on Github, hosted on Railway, no CICD 
- Simple user authentication using JWT
- PostgreSQL DB to store user credentials and chat data
- Simple chatbot using OpenAI GPT API
- Basic chat session management that allow creating new or selecting previous chat seesions

Show that you understand the overview and current status. Next prompt, I will brief you detai

In [17]:
# current_task = """
# - Add a scrollable list displaying all chat session from a given user
# - Within the list, display the name of each chat session
# - Sort the list by the last opened time of a chat session to display latest opened session.
# """

current_task = """
- Add localization support for English and Simplified Chinese
"""


steps = """
- Refine and articulate features
- Design logic for the features
- Divide into implementation steps
"""


prompt_2 = f"""Current task{current_task}
I need your help in three things {steps}
Let's start with 'Refine and articulate features', discuss the features with me. Once we agree on the features, we can proceed to design logic
"""


print(prompt_2)

Current task
- Add localization support for English and Simplified Chinese

I need your help in three things 
- Refine and articulate features
- Design logic for the features
- Divide into implementation steps

Let's start with 'Refine and articulate features', discuss the features with me. Once we agree on the features, we can proceed to design logic



In [28]:
prompt_3 = f"""Perfect! You read my mind! Now, let's design the logic together. 
I believe knowledge on current file structure and brief descriptions for key scripts can help you understand the current logic, so please read this carefully
{file_descriptions}
Now please discuss the logic design with me. Once we agree on the logics, we can proceed to divide into implementation steps
"""


print(prompt_3)

Perfect! You read my mind! Now, let's design the logic together. 
I believe knowledge on current file structure and brief descriptions for key scripts can help you understand the current logic, so please read this carefully
- flask/
    - requirements.txt
    - database.py: Initializes an SQLAlchemy database instance for the application to interact with the database.
    - model.py: Defines the structure of the database tables for the chat application, including Users, ChatSessions, ChatMessages, Chatbots, and the relationships between them.
    - main.py: a Flask server Script that handles user registration, login, and interactive chat sessions using SocketIO, with a chatbot utilizing GPT for generating responses.
    - simple_gpt_chatbot.py: Interfaces with OpenAI's GPT-3.5 model to generate responses for a chatbot, using the historical data of a chat session retrieved from a database.
- web_chat/
    - package-lock.json
    - package.json
    - src/
        - reportWebVitals.js
    

In [ ]:
implementation_steps = """
Update main.py and model.py in the Flask server to handle fetching user chat sessions and chat history.
Create the ChatSessionList.js and ChatSessionItem.js React components to display and manage the chat session list.
Implement fetching and sorting of user chat sessions in ChatSessionList.js.
Add click event handling to ChatSessionItem.js and manage session selection state in the React application.
Update the Chat.js component to fetch and display the chat history of the selected session.

"""

In [31]:
import datetime

def write_prompts_to_file():
    # Get the current date and time
    now = datetime.datetime.now()
    
    # Format the date and time as a string for the file name
    file_name = now.strftime("%Y-%m-%d_%H-%M-%S.txt")
    
    # Open the file in write mode
    with open(os.path.join("logs", file_name), 'w') as file:
        # Write the string to the file
        file.write("Prompt1")
        file.write(prompt_1)  
        file.write(f"\n\n")  
        file.write("Prompt2")  
        file.write(f"\n\n") 
        file.write(prompt_2)        
        
    print(f'String written to file: {file_name}')
    
write_prompts_to_file()

String written to file: 2023-05-17_13-46-52.txt
